<a href="https://colab.research.google.com/github/PiotrMaciejKowalski/BigData2022-actors/blob/Policzenie-pierwszych-similarity/colabs/Liczenie_similarity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Przygotowanie notatnika

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')
import pandas as pd

Mounted at /content/gdrive


In [2]:
!git clone https://github.com/PiotrMaciejKowalski/BigData2022-actors.git
!mv /content/BigData2022-actors/* .
!mv /content/BigData2022-actors/.* .
!rmdir /content/BigData2022-actors/

Cloning into 'BigData2022-actors'...
remote: Enumerating objects: 1182, done.
remote: Counting objects: 100% (346/346), done.
remote: Compressing objects: 100% (175/175), done.
remote: Total 1182 (delta 264), reused 182 (delta 171), pack-reused 836
Receiving objects: 100% (1182/1182), 5.88 MiB | 29.66 MiB/s, done.
Resolving deltas: 100% (666/666), done.
mv: cannot move '/content/BigData2022-actors/.' to './.': Device or resource busy
mv: cannot move '/content/BigData2022-actors/..' to './..': Device or resource busy


In [3]:
!git checkout Policzenie-pierwszych-similarity

Branch 'Policzenie-pierwszych-similarity' set up to track remote branch 'Policzenie-pierwszych-similarity' from 'origin'.
Switched to a new branch 'Policzenie-pierwszych-similarity'


## Załadowanie danych z pliku o rozszerzeniu .parquet

In [4]:
from lib.const import JOINED_DATA, DATA_WITH_AWARDS

W poradniku "Tworzenie skrotu do GDrive" w folderze tutorials jest poradnik jak sprawić żeby poniższa lokalizacja się poprawnie ładowała.


In [5]:
data = pd.read_parquet(JOINED_DATA)
# TODO przepiąć się na nowszy plik gdy będzie on już zapisany na dysku

# Liczenie similarity

In [6]:
from lib.similarity_utils import *

## Szukanie aktorów na przykładzie Sylvestera Stallone i Arnolda Schwarzeneggera

Za pomocą zdefiniowanych metod jesteśmy w stanie odnaleźć wskazanego aktora w bazie danych na podstawie strony [IMDB](https://www.imdb.com/?ref_=nv_home). Wystarczy odnaleźć na stronie wybranego przez siebie aktora, a następnie z linku skopiować jego id. Id aktora składa się kolejno z liter "nm", a następnie $6$ lub $7$ cyfr. Przykładowe id aktora znajduje się w kodzie poniżej.

In [7]:
Sylvester_Stallone = find_actor(data, 'nm0000230')

In [8]:
Sylvester_Stallone

nconst                                                    nm0000230
tconst            [tt0083284, tt0132245, tt0160184, tt1308729, t...
titleType         [movie, movie, movie, movie, tvEpisode, tvEpis...
originalTitle     [Victory, Driven, D-Tox, Bullet to the Head, C...
isAdult           [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ...
startYear                                                      1969
endYear                                                          \N
genres            [Drama,Sport,War, Action,Drama,Sport, Crime,Th...
category                                                      actor
characters        [["The Players: U.S.A. - Capt. Robert Hatch"],...
primaryName                                      Sylvester Stallone
knownForTitles              tt0075148,tt3076658,tt0089927,tt0084602
Name: 1949571, dtype: object

In [9]:
Arnold_schwarzenegger = find_actor(data, 'nm0000216')

In [10]:
Arnold_schwarzenegger

nconst                                                    nm0000216
tconst            [tt0233469, tt0364056, tt0711516, tt15739442, ...
titleType         [movie, videoGame, tvEpisode, video, tvEpisode...
originalTitle     [Collateral Damage, Terminator 3: Rise of the ...
isAdult           [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
startYear                                                      1970
endYear                                                          \N
genres            [Action,Drama,Thriller, Action,Adventure,Sci-F...
category                                                      actor
characters        [["Gordy Brewer"], ["The Terminator"], ["Josef...
primaryName                                   Arnold Schwarzenegger
knownForTitles              tt0088247,tt0107362,tt0216216,tt0103064
Name: 1639999, dtype: object

## Similarity dla pary aktorów

Za pomocą metody `similarity` jesteśmy w stanie policzyć podobieństwo dwóch wybranych aktorów. Przed przystąpieniem do jego liczenia, trzeba najpierw zamienić dane o aktorach w pythonowe listy.


In [11]:
Sylvester_Stallone = prepare_pandas_row(Sylvester_Stallone)
Arnold_schwarzenegger = prepare_pandas_row(Arnold_schwarzenegger)

Z gotowymi listami przystepujemy do liczenia podobieństwa pomiędzy aktorami.

In [12]:
print(f"Podobieństwo pomiędzy {Sylvester_Stallone[10]} i {Arnold_schwarzenegger[10]} wynosi\
 {round(similarity(Sylvester_Stallone, Arnold_schwarzenegger), 3)}.")

Podobieństwo pomiędzy Sylvester Stallone i Arnold Schwarzenegger wynosi -0.199.


## Szukanie najbardziej podobnych aktorów

Za pomocą metody `similarity_one_vs_all` liczymy podobieństwa pomiędzy jednym aktorem, a resztą aktorów z całego naszego zbioru danych. Najpierw znajdziemy $3$ aktorów najpodobniejszych do Sylvestera Stallone.


### Sylvester Stallone

In [13]:
ids1, values1 = similarity_one_vs_all(data, Sylvester_Stallone)
values1, ids1 = sort_two_lists(values1, ids1, reverse=True)

In [14]:
print_most_similiar_actors(data, Sylvester_Stallone, ids1, values1)

Najbardziej podobnymi do Sylvester Stallone aktorami/aktorkami są w kolejności:
  - Bruce Willis z similarity równym: -0.126
  - Logan Marshall-Green z similarity równym: -0.131
  - Matthew Yang King z similarity równym: -0.136


W podobny sposób co poprzednio, teraz będziemy szukać $3$ aktorów najpodobniejszych do Arnolda Schwarzeneggera, Jennifer Lawerence oraz Natalie Portman.

### Arnold Schwarzenegger

In [15]:
ids2, values2 = similarity_one_vs_all(data, Arnold_schwarzenegger)
values2, ids2 = sort_two_lists(values2, ids2, reverse=True)

In [16]:
print_most_similiar_actors(data, Arnold_schwarzenegger, ids2, values2)

Najbardziej podobnymi do Arnold Schwarzenegger aktorami/aktorkami są w kolejności:
  - Bill Paxton z similarity równym: -0.105
  - Jeff Goldblum z similarity równym: -0.106
  - Michael Keaton z similarity równym: -0.107


### Jennifer Lawrence

In [17]:
Jennifer_Lawrence = prepare_pandas_row(find_actor(data, 'nm2225369'))

In [18]:
ids3, values3 = similarity_one_vs_all(data, Jennifer_Lawrence)
values3, ids3 = sort_two_lists(values3, ids3, reverse=True)

In [19]:
print_most_similiar_actors(data, Jennifer_Lawrence, ids3, values3)

Najbardziej podobnymi do Jennifer Lawrence aktorami/aktorkami są w kolejności:
  - Isabel Martínez 'La Tarabilla' z similarity równym: -0.096
  - Angel Aviles z similarity równym: -0.1
  - Gogo Antzoletaki z similarity równym: -0.104


### Natalie Portman

In [20]:
Natalie_Portman = prepare_pandas_row(find_actor(data, 'nm0000204'))

In [21]:
ids4, values4 = similarity_one_vs_all(data, Natalie_Portman)
values4, ids4 = sort_two_lists(values4, ids4, reverse=True)

In [22]:
print_most_similiar_actors(data, Natalie_Portman, ids4, values4)

Najbardziej podobnymi do Natalie Portman aktorami/aktorkami są w kolejności:
  - Heeba Shah z similarity równym: -0.129
  - Aleksa Palladino z similarity równym: -0.13
  - Madison Fitts z similarity równym: -0.131
